<a href="https://colab.research.google.com/github/carlnaces/test/blob/main/Marketing_AS_LM_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
# Install the required libraries
!pip install gspread google-auth

# Import libraries
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default

# Authenticate and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [203]:
import pandas as pd
import re
import numpy as np

# Open the Google Sheets file (replace 'your_spreadsheet_name' with your file's name)
spreadsheet = gc.open("Marketing AS to LM").get_worksheet(0)
spreadsheet2 = gc.open("Marketing AS to LM").get_worksheet(1)
spreadsheet3 = gc.open("Marketing AS to LM").get_worksheet(2)

# Now you can work with the 'worksheet' object, for example, fetching the data
data = spreadsheet.get_all_values()
data2 = spreadsheet2.get_all_values()
data3 = spreadsheet3.get_all_values()

#Create Data frames for each sheet and assign it to Keap, Acuity and Actiostep data
action = pd.DataFrame(data)
lm = pd.DataFrame(data2)
keap = pd.DataFrame(data3)

#Make each first row as the column headers
action.columns = action.iloc[0]
lm.columns = lm.iloc[0]
keap.columns = keap.iloc[0]

action = action.drop(0)
lm = lm.drop(0)
keap = keap.drop(0)

In [ ]:
#print(action.head(10))
#print(lm.head(10))

# Preparing the Actionstep data

In [204]:
#Prep Actionstep Data
# Replace carriage return with comma and trim spaces in the 'Client Email' column
action['Client Email'] = action['Client Email'].str.replace('\n', ',').str.strip()

# Assuming df is your DataFrame and 'Client Email' is the column containing the email addresses
action['Client Email'] = action['Client Email'].str.replace('_\|\|_', '')

# Assuming df is your DataFrame and 'Client Email' is the column containing the email addresses
action['Client Email'] = action['Client Email'].str.strip('_')

action['Client Email'] = action['Client Email'].str.replace('__', ',')

# Make Emails Lowecase
action['Client Email'] = action['Client Email'].str.lower()

In [205]:
# Convert 'Client Email' and 'Matter Type' columns to string data type
action[['Client Email', 'Matter Type']] = action[['Client Email', 'Matter Type']].astype(str)

# List of columns to convert from string to numeric
columns_to_convert = ['Fee Collected (APC)', 'Fee Collected (EP)', 'Fee Collected (EA)', 'Fee Collected (TA)', 'Fee Collected (Old EP)']

# Remove dollar signs ($) and commas (,) from the values, convert to numeric, fill NaNs with 0, and ensure two decimal places
for column in columns_to_convert:
    action[column] = pd.to_numeric(action[column].str.replace('[$,]', '', regex=True), errors='coerce').fillna(0).astype(float)

# Round the converted columns to ensure two decimal places
action[columns_to_convert] = action[columns_to_convert].round(2)

# Create 'Max Fee' column by taking the maximum value across the specified columns and rounding it to two decimal places
action['Max Fee'] = action[columns_to_convert].max(axis=1).round(2)

# Create a new DataFrame by dropping specified columns
action_v2 = action.drop(columns=['Client Last Name', 'Client First Name',
                                 'Fee Collected (APC)', 'Fee Collected (EP)',
                                 'Fee Collected (EA)', 'Fee Collected (TA)',
                                 'Fee Collected (Old EP)'])

# Convert 'ID' column to integer
action_v2['ID'] = action_v2['ID'].astype(int)

# Sort the new DataFrame by 'ID' column in ascending order
action_check_sorted = action_v2.sort_values(by='ID', ascending=True)

# Replace empty strings in 'Client Email' with NaN and drop rows with NaN in 'Client Email'
action_v2['Client Email'] = action_v2['Client Email'].replace('', pd.NA)
action_v2 = action_v2.dropna(subset=['Client Email'])

In [206]:
print(action_v2.dtypes)
action_v2.to_excel('action_v2.xlsx', index=False)

0
Favorite         object
Client Email     object
ID                int64
Date Created     object
Matter Type      object
Max Fee         float64
dtype: object


In [207]:
# Split the 'Client Email split' column into a list of emails
action_v2['Client Email List'] = action_v2['Client Email'].str.split(',')

# Create a new DataFrame with one row for each email in 'Client Email List'
expanded_action_v2 = action_v2.explode('Client Email List')

#sort by ID
expanded_action_v2 = expanded_action_v2.sort_values(by='ID')

columns_to_sort_by = ['ID', 'Max Fee', 'Client Email List']
expanded_action_v2 = expanded_action_v2.sort_values(by=columns_to_sort_by, ascending=[True, False, True])

columns_to_check = ['ID', 'Date Created', 'Matter Type', 'Client Email', 'Max Fee', 'Client Email List']
expanded_action_v2 = expanded_action_v2.drop_duplicates(subset=columns_to_check, keep='first')

In [208]:
expanded_action_v2.to_excel('expanded_action_v2.xlsx', index=False)

# Splitting into Primary and Secondary Emails

In [209]:
#Removing Duplicates
# Find the first entry for each ID
first_entry_mask = expanded_action_v2.duplicated(subset='ID')

# Filter out the first entry for each ID (primary email)
primary_email = expanded_action_v2[~first_entry_mask]

# Filter the DataFrame to exclude the rows included in primary_email (secondary email)
secondary_email = expanded_action_v2[first_entry_mask]

In [210]:
primary_email.to_excel('primary_email.xlsx', index=False)
secondary_email.to_excel('secondary_email.xlsx', index=False)

# Prepare Lawmatics Export Data

In [211]:
#Clean LM columns first

selected_columns = ['id',
                    'email',
                    'practice_area_name',
                    'source_name',
                    'Requested Understanding Estate Administration(matter)',
                    'Vision Meeting(matter)',
                    'Strategy Session(matter)',
                    'Actionstep Matter ID(matter)',
                    'Discovery Meeting(matter)',
                    'Keap Lead Source(matter)',
                    'Requested Caregiver Guide(matter)',
                    'Requested CSP Guide(matter)',
                    'Requested 12 Threats(matter)',
                    "Requested Alzheimer's Guide(matter)",
                    'Requested COVID-19 Economic Relief & Incentives for Small Employers Report(matter)',
                    'Requested Hardcopy of CSP Guide(matter)',
                    'Requested How to Choose(matter)',
                    'Requested Choosing Guardians(matter)',
                    'Requested Special Needs Guide(matter)',
                    "Requested Here's the Plan(matter)",
                    'Requested Hardcopy of Caregivers Guide(matter)',
                    'Requested Shook 1 Action Plan(matter)',
                    "Reqeusted Parkinson's Peace of Mind(matter)",
                    'Requested Avoid Nursing Home Poverty(matter)',
                    'Requested Shook 2 Action Plan(matter)',
                    'Requested Hardcopy of 12 Threats(matter)',
                    'Requested Supercharge Your IRA(matter)',
                    'Requested Understanding VA A&A(matter)',
                    'Requested Estate Planning and Coronavirus(matter)',
                    'Requested First Steps Checklist(matter)',
                    'Requested SECURE Act Memo(matter)',
                    "Requested Business Owner's Guide to Surviving COVID-19(matter)",
                    'Requested CFEP Personal Organizer(matter)',
                    'Requested Hardcopy Newsletter(matter)',
                    'Requested Hardcopy of Understanding Estate Administration(matter)',
                    'created_at']

# Method 1: Overwrite the DataFrame with selected columns
lm_cleaned = lm[selected_columns]

# Clean column names to remove leading/trailing spaces
lm_cleaned.columns = lm_cleaned.columns.str.strip()

# List of columns to convert to string, using double quotes to handle apostrophes
columns_to_convert = ['email',
                    'practice_area_name',
                    'source_name',
                    'Keap Lead Source(matter)']

# Check which columns are actually present in the DataFrame
existing_columns_to_convert = [col for col in columns_to_convert if col in lm_cleaned.columns]

In [212]:
# Convert the existing columns to string data type
if existing_columns_to_convert:
    lm_cleaned[existing_columns_to_convert] = lm_cleaned[existing_columns_to_convert].astype(str)
else:
    print("None of the specified columns exist in the DataFrame.")

# List of columns to convert to int
columns_to_convert_to_int = [
    'id', 'Actionstep Matter ID(matter)'
]

# Check if the specified columns exist in the DataFrame and handle them accordingly
existing_columns_to_convert = [col for col in columns_to_convert_to_int if col in lm_cleaned.columns]

if existing_columns_to_convert:
    # Convert columns to numeric, coerce errors to NaN, fill NaNs with a default value (e.g., 0), and convert to int
    lm_cleaned[existing_columns_to_convert] = lm_cleaned[existing_columns_to_convert].apply(
        pd.to_numeric, errors='coerce').fillna(0).astype(int)
else:
    print("None of the specified columns exist in the DataFrame.")

# List of columns to convert to date
columns_to_convert_to_date = [
                    'Requested Understanding Estate Administration(matter)',
                    'Vision Meeting(matter)',
                    'Strategy Session(matter)',
                    'Discovery Meeting(matter)',
                    'Requested Caregiver Guide(matter)',
                    'Requested CSP Guide(matter)',
                    'Requested 12 Threats(matter)',
                    "Requested Alzheimer's Guide(matter)",
                    'Requested COVID-19 Economic Relief & Incentives for Small Employers Report(matter)',
                    'Requested Hardcopy of CSP Guide(matter)',
                    'Requested How to Choose(matter)',
                    'Requested Choosing Guardians(matter)',
                    'Requested Special Needs Guide(matter)',
                    "Requested Here's the Plan(matter)",
                    'Requested Hardcopy of Caregivers Guide(matter)',
                    'Requested Shook 1 Action Plan(matter)',
                    "Reqeusted Parkinson's Peace of Mind(matter)",
                    'Requested Avoid Nursing Home Poverty(matter)',
                    'Requested Shook 2 Action Plan(matter)',
                    'Requested Hardcopy of 12 Threats(matter)',
                    'Requested Supercharge Your IRA(matter)',
                    'Requested Understanding VA A&A(matter)',
                    'Requested Estate Planning and Coronavirus(matter)',
                    'Requested First Steps Checklist(matter)',
                    'Requested SECURE Act Memo(matter)',
                    "Requested Business Owner's Guide to Surviving COVID-19(matter)",
                    'Requested CFEP Personal Organizer(matter)',
                    'Requested Hardcopy Newsletter(matter)',
                    'Requested Hardcopy of Understanding Estate Administration(matter)',
                    'created_at']

# Check if the specified columns exist in the DataFrame and handle them accordingly
existing_columns_to_convert = [col for col in columns_to_convert_to_date if col in lm_cleaned.columns]

if existing_columns_to_convert:
    # Convert columns to datetime, coerce errors to NaT (Not a Time)
    lm_cleaned[existing_columns_to_convert] = lm_cleaned[existing_columns_to_convert].apply(
        pd.to_datetime, errors='coerce')
else:
    print("None of the specified columns exist in the DataFrame.")

<ipython-input-212-6742869b3043>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lm_cleaned[existing_columns_to_convert] = lm_cleaned[existing_columns_to_convert].astype(str)
<ipython-input-212-6742869b3043>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lm_cleaned[existing_columns_to_convert] = lm_cleaned[existing_columns_to_convert].apply(
<ipython-input-212-6742869b3043>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [213]:
#print(lm_cleaned.dtypes)
lm_cleaned.to_excel('lm_cleaned.xlsx', index=False)

In [214]:
# Use .loc to avoid SettingWithCopyWarning
lm_cleaned.loc[:, 'Had Vision'] = lm_cleaned['Vision Meeting(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
lm_cleaned.loc[:, 'Had Strategy'] = lm_cleaned['Strategy Session(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
lm_cleaned.loc[:, 'Had Discovery'] = lm_cleaned['Discovery Meeting(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)

<ipython-input-214-0692c624b940>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lm_cleaned.loc[:, 'Had Vision'] = lm_cleaned['Vision Meeting(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
<ipython-input-214-0692c624b940>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lm_cleaned.loc[:, 'Had Strategy'] = lm_cleaned['Strategy Session(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
<ipython-input-214-0692c624b940>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [215]:
# Convert the date columns to datetime format
date_columns = ['Requested Understanding Estate Administration(matter)',
                    'Requested Caregiver Guide(matter)',
                    'Requested CSP Guide(matter)',
                    'Requested 12 Threats(matter)',
                    "Requested Alzheimer's Guide(matter)",
                    'Requested COVID-19 Economic Relief & Incentives for Small Employers Report(matter)',
                    'Requested Hardcopy of CSP Guide(matter)',
                    'Requested How to Choose(matter)',
                    'Requested Choosing Guardians(matter)',
                    'Requested Special Needs Guide(matter)',
                    "Requested Here's the Plan(matter)",
                    'Requested Hardcopy of Caregivers Guide(matter)',
                    'Requested Shook 1 Action Plan(matter)',
                    "Reqeusted Parkinson's Peace of Mind(matter)",
                    'Requested Avoid Nursing Home Poverty(matter)',
                    'Requested Shook 2 Action Plan(matter)',
                    'Requested Hardcopy of 12 Threats(matter)',
                    'Requested Supercharge Your IRA(matter)',
                    'Requested Understanding VA A&A(matter)',
                    'Requested Estate Planning and Coronavirus(matter)',
                    'Requested First Steps Checklist(matter)',
                    'Requested SECURE Act Memo(matter)',
                    "Requested Business Owner's Guide to Surviving COVID-19(matter)",
                    'Requested CFEP Personal Organizer(matter)',
                    'Requested Hardcopy Newsletter(matter)',
                    'Requested Hardcopy of Understanding Estate Administration(matter)']

# Create 'First Lead' column by finding the earliest date
lm_cleaned['First Lead'] = lm_cleaned[date_columns].apply(lambda row: row.idxmin() if row.notna().any() else pd.NA, axis=1)

<ipython-input-215-57c456838756>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lm_cleaned['First Lead'] = lm_cleaned[date_columns].apply(lambda row: row.idxmin() if row.notna().any() else pd.NA, axis=1)


In [216]:
# Sample data creation for demonstration, replace this with your actual data loading method
# lm_cleaned = pd.read_csv('path_to_your_csv_file.csv') or however you load your DataFrame

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
lm_cleaned_copy = lm_cleaned.copy()

# Display initial data for debugging
print("Initial Data Sample:")
print(lm_cleaned_copy[['email', 'source_name', 'Keap Lead Source(matter)']].head(50))

# Check for null values and empty strings in 'source_name'
print("\nChecking for null or empty values in 'source_name':")
null_values = lm_cleaned_copy['source_name'].isnull().sum()
empty_strings = (lm_cleaned_copy['source_name'].str.strip() == "").sum()
print(f"Null values found: {null_values}")
print(f"Empty strings found: {empty_strings}")

# Function to update 'Keap Lead Source(matter)'
def update_keap_lead_source(row):
    if pd.notnull(row['source_name']) and row['source_name'].strip() != "":
        return row['source_name']
    else:
        return row['Keap Lead Source(matter)']

# Apply the function to update 'Keap Lead Source(matter)' column
lm_cleaned_copy['Keap Lead Source(matter)'] = lm_cleaned_copy.apply(update_keap_lead_source, axis=1)

# Display updated data for verification
print("\nUpdated Data Sample:")
print(lm_cleaned_copy[['email', 'source_name', 'Keap Lead Source(matter)']].head(50))

# Save the updated DataFrame to an Excel file
lm_cleaned_copy.to_excel('updated_lm_cleaned.xlsx', index=False)


Initial Data Sample:
0                                         email     source_name  \
1                     aliceboone61921@gmail.com                   
2                          eroseville@yahoo.com                   
3                                 lcarm@att.net                   
4                              jvereen@atmc.net                   
5                       akendricks@netscape.net                   
6                          pdycornell@gmail.com                   
7                         alyssah.o13@gmail.com                   
8                          mscandi_99@yahoo.com                   
9                        thomashstyer@gmail.com                   
10                         tonyrice60@gmail.com                   
11                           cwimes02@gmail.com                   
12                                                                
13                      pauldowsett@hotmail.com                   
14                     dalestasiewicz@gma

In [217]:
# List of columns to keep
columns_to_keep = ['id',
                    'email',
                    'practice_area_name',
                    'Vision Meeting(matter)',
                    'Strategy Session(matter)',
                    'Actionstep Matter ID(matter)',
                    'Discovery Meeting(matter)',
                    'Keap Lead Source(matter)',
                    'created_at',
                    'Had Vision',
                   'Had Strategy',
                   'Had Discovery',
                   'First Lead']

# Drop all other columns
lm_cleaned = lm_cleaned_copy[columns_to_keep]

In [218]:
# Sort by 'created_at' and then by 'email'
lm_cleaned = lm_cleaned.sort_values(by=['created_at', 'email'])

In [219]:
# Get unique values of 'Matter Type' column
unique_matter_types = primary_email['Matter Type'].unique()

# Create an empty dictionary to store filtered DataFrames
filtered_dataframes = {}

# Split primary_email into multiple DataFrames based on 'Matter Type'
for matter_type in unique_matter_types:
    # Remove spaces from matter type for DataFrame name
    dataframe_name = matter_type.replace(' ', '_')
    filtered_dataframes[dataframe_name] = primary_email[primary_email['Matter Type'] == matter_type]

In [ ]:
# #CHECKING
# # Now check the contents of the filtered DataFrames
# for dataframe_name, dataframe in filtered_dataframes.items():
#     print(f"\nDataFrame for Matter Type '{dataframe_name}':")
#     print(dataframe)

# # Get the names of the new DataFrames
# new_dataframe_names = list(filtered_dataframes.keys())

# # Print the names of the new DataFrames
# print("Names of the new DataFrames:")
# for name in new_dataframe_names:
#     print(name)

In [220]:
# Assign the dataframe for 'CFEP Estate Planning' to CFEP_Estate_Planning
CFEP_Estate_Planning = filtered_dataframes['CFEP_Estate_Planning']
CFEP_Estate_Administration = filtered_dataframes['CFEP_Estate_Administration']
CFEP_Guardianship = filtered_dataframes['CFEP_Guardianship']
CFEP_Medicaid = filtered_dataframes['CFEP_Medicaid']
CFEP_Trust_Administration = filtered_dataframes['CFEP_Trust_Administration']
APC_Planning = filtered_dataframes['APC_Planning']

In [221]:
# Save each DataFrame to an Excel file
for name, dataframe in filtered_dataframes.items():
    filename = f"{name}.xlsx"
    dataframe.to_excel(filename, index=False)
    print(f"DataFrame saved to '{filename}'.")

DataFrame saved to 'CFEP_Estate_Planning.xlsx'.
DataFrame saved to 'CFEP_Trust_Administration.xlsx'.
DataFrame saved to 'CFEP_Medicaid.xlsx'.
DataFrame saved to 'CFEP_Estate_Administration.xlsx'.
DataFrame saved to 'CFEP_Guardianship.xlsx'.
DataFrame saved to 'APC_Planning.xlsx'.


In [222]:
#print(lm_cleaned.dtypes)
lm_cleaned.to_excel('lm_cleaned.xlsx', index=False)

Step by step merge done here. This step is for CFEP Estate Planning.


In [223]:
# Filter lm_cleaned for 'Estate Planning' and 'Estate Planning Update' practice areas
lm_cleaned_filtered = lm_cleaned[(lm_cleaned['practice_area_name'] == 'Estate Planning') | (lm_cleaned['practice_area_name'] == 'Estate Planning Update')]

# Merge lm_cleaned_filtered with CFEP Estate Planning dataframe using left join
merged_df1 = pd.merge(lm_cleaned_filtered, CFEP_Estate_Planning.drop_duplicates('Client Email List'), left_on='email', right_on='Client Email List', how='left', indicator=True)

# Merge lm_cleaned with CFEP Estate Planning dataframe using left join
merged_df2 = pd.merge(lm_cleaned, CFEP_Estate_Planning.drop_duplicates('Client Email List'), left_on='email', right_on='Client Email List', how='left', indicator=True)

# Filter rows that were not merged in merged_df2
unmerged_df2 = merged_df2[merged_df2['_merge'] == 'left_only']

# Concatenate the merged dataframes
# FINAL TABLE HERE FOR CFEP ESTATE PLANNING
final_merged_df = pd.concat([merged_df1, unmerged_df2]).drop_duplicates()

# Save the merged dataframe to an Excel file
final_merged_df.to_excel('final_merged_data.xlsx', index=False)

# Filter rows from CFEP_Estate_Planning that were not merged
unmatched_CFEP_Estate_Planning = CFEP_Estate_Planning[~CFEP_Estate_Planning['Client Email List'].isin(final_merged_df['Client Email List'])]

# Save the unmatched rows to a new dataframe
unmatched_CFEP_Estate_Planning.to_excel('unmatched_CFEP_Estate_Planning.xlsx', index=False)


In [224]:
# Filter final_merged_df for 'Estate Administration' practice area
final_merged_estate_administration = final_merged_df[final_merged_df['practice_area_name'] == 'Estate Administration']

# Merge final_merged_estate_administration with CFEP Estate Administration dataframe using left join
#merged_df_estate_administration = pd.merge(final_merged_estate_administration, CFEP_Estate_Administration, left_on='email', right_on='Client Email List', how='left')

# Merge final_merged_estate_administration with CFEP Estate Administration dataframe using left join
merged_df_estate_administration = pd.merge(final_merged_estate_administration, CFEP_Estate_Administration, left_on='email', right_on='Client Email List', how='left', indicator='_merge_indicator')

# Save the merged dataframe to an Excel file
merged_df_estate_administration.to_excel('merged_estate_administration.xlsx', index=False)

# Filter final_merged_df for rows not in 'Estate Administration' practice area
unmatched_rows = final_merged_df[final_merged_df['practice_area_name'] != 'Estate Administration']

# Save the unmatched rows to an Excel file
unmatched_rows.to_excel('unmatched_rows.xlsx', index=False)

# Merge merged_df_estate_administration with unmatched_rows
merged_df_with_unmatched = pd.concat([merged_df_estate_administration, unmatched_rows])

# Save the merged dataframe to an Excel file
## FINAL TABLE FOR ESTATE PLANNING AND ESTATE ADMINISTRATION
merged_df_with_unmatched.to_excel('FINAL_EP_EA.xlsx', index=False)

# Filter rows from CFEP_Estate_Administration that were not merged
unmatched_CFEP_Estate_Administration = CFEP_Estate_Administration[~CFEP_Estate_Administration['Client Email List'].isin(merged_df_with_unmatched['Client Email List_y'])]

# Save the unmatched rows to a new dataframe
unmatched_CFEP_Estate_Administration.to_excel('unmatched_CFEP_Estate_Administration.xlsx', index=False)



In [225]:
#TAKE NOTE OF UNMERGED DATA

unmatched_CFEP = pd.concat([unmatched_CFEP_Estate_Planning,
                            unmatched_CFEP_Estate_Administration,
                            CFEP_Medicaid,
                            CFEP_Trust_Administration,
                            CFEP_Guardianship,
                            APC_Planning])

unmatched_CFEP.to_excel('unmatched_CFEP.xlsx', index=False)

In [226]:
#CLEAN FINAL EP AND EA Dataframe.
# TAKE NOTE AS WELL of the remaining rows from secondary_email.

# Move values from 'Vision Meeting(matter)' to 'Discovery Meeting(matter)'
condition = (merged_df_with_unmatched['practice_area_name'] == 'Estate Administration') & (merged_df_with_unmatched['Vision Meeting(matter)'].notnull())

# Update 'Discovery Meeting(matter)' column with the values from 'Vision Meeting(matter)'
merged_df_with_unmatched.loc[condition, 'Discovery Meeting(matter)'] = merged_df_with_unmatched.loc[condition, 'Vision Meeting(matter)']

# Set 'Vision Meeting(matter)' column to NaN or empty after moving the values
merged_df_with_unmatched.loc[condition, 'Vision Meeting(matter)'] = None  # or use np.nan for NaN values


# Use .loc to avoid SettingWithCopyWarning
merged_df_with_unmatched.loc[:, 'Had Vision'] = merged_df_with_unmatched['Vision Meeting(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
merged_df_with_unmatched.loc[:, 'Had Strategy'] = merged_df_with_unmatched['Strategy Session(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)
merged_df_with_unmatched.loc[:, 'Had Discovery'] = merged_df_with_unmatched['Discovery Meeting(matter)'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Check the final state of the dataframe to confirm the changes
#print(merged_df_with_unmatched.head())

# Save the modified dataframe to an Excel file
merged_df_with_unmatched.to_excel('FINAL_FINAL_EP_EA.xlsx', index=False)


In [227]:
# List of columns to keep
columns_to_keep = [ 'email',
                    'practice_area_name',
                    'Vision Meeting(matter)',
                    'Strategy Session(matter)',
                    'Actionstep Matter ID(matter)',
                    'Discovery Meeting(matter)',
                    'Keap Lead Source(matter)',
                    'created_at',
                    'Had Vision',
                   'Had Strategy',
                   'Had Discovery',
                   'First Lead',
                    'ID_y',
                    'Date Created_y',
                    'Max Fee_y',
                    'ID',
                    'Date Created',
                    'Max Fee']

# Drop all other columns
merged_df_with_unmatched = merged_df_with_unmatched[columns_to_keep]

# Save the modified dataframe to an Excel file
merged_df_with_unmatched.to_excel('FINAL_FINAL_EP_EA.xlsx', index=False)

In [228]:
#Clean email addresses
# Define the unwanted email values
unwanted_emails = ["", None, "none@example.com", "none@none.com", 'none']

# Remove rows where 'email' is in the unwanted emails list
cleaned_df = merged_df_with_unmatched[~merged_df_with_unmatched['email'].isin(unwanted_emails)]

# Optionally, you can also remove rows with completely blank email values that might not be caught by the above
cleaned_df = cleaned_df[cleaned_df['email'].str.strip() != ""]


# Save the cleaned DataFrame to an Excel file
cleaned_df.to_excel('cleaned_merged_df_with_unmatched.xlsx', index=False)


In [229]:
# Combine 'ID_y' and 'ID' into a single 'ID' column
cleaned_df['ID'] = cleaned_df['ID_y'].combine_first(cleaned_df['ID'])

# Combine 'Max Fee_y' and 'Max Fee' into a single 'Max Fee' column
cleaned_df['Max Fee'] = cleaned_df['Max Fee_y'].combine_first(cleaned_df['Max Fee'])

# Combine 'Max Fee_y' and 'Max Fee' into a single 'Max Fee' column
cleaned_df['Date Created'] = cleaned_df['Date Created_y'].combine_first(cleaned_df['Date Created'])

# Drop the old columns
cleaned_df.drop(columns=['ID_y', 'Max Fee_y','Date Created_y'], inplace=True)

# Save the cleaned DataFrame to an Excel file
cleaned_df.to_excel('cleaned_merged_df_with_unmatched.xlsx', index=False)

In [230]:
# List of columns to keep
columns_to_keep = ['Email',
                   'Leadsource']

# Drop all other columns
keap = keap[columns_to_keep]

# Save the modified dataframe to an Excel file
keap.to_excel('keap.xlsx', index=False)

In [231]:
# Merge final_merged_estate_administration with CFEP Estate Administration dataframe using left join
merged_df_FINAL = pd.merge(cleaned_df, keap, left_on='email', right_on='Email', how='left')

#merged_df_FINAL['Leadsource'] = merged_df_FINAL.apply(lambda x: x['Leadsource'] if pd.notnull(x['Keap Lead Source(matter)']) else x['Keap Lead Source(matter)'], axis=1)

merged_df_FINAL['Leadsource'].fillna(merged_df_FINAL['Keap Lead Source(matter)'], inplace=True)

# Save the cleaned DataFrame to an Excel file
merged_df_FINAL.to_excel('merged_df_FINAL.xlsx', index=False)

In [232]:

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
merged_df_FINAL_copy = merged_df_FINAL.copy()

# Function to update 'Keap Lead Source(matter)'
def update_keap_lead_source(row):
    if pd.notnull(row['Keap Lead Source(matter)']) and row['Keap Lead Source(matter)'].strip() != "":
        return row['Keap Lead Source(matter)']
    else:
        return row['Leadsource']

# Apply the function to update 'Keap Lead Source(matter)' column
merged_df_FINAL_copy['Leadsource'] = merged_df_FINAL_copy.apply(update_keap_lead_source, axis=1)


# Save the updated DataFrame to an Excel file
merged_df_FINAL_copy.to_excel('merged_df_FINAL_copy.xlsx', index=False)

In [234]:
# List of columns to keep
columns_to_keep = [ 'email',
                    'practice_area_name',
                    'Vision Meeting(matter)',
                    'Strategy Session(matter)',
                    'Actionstep Matter ID(matter)',
                    'Discovery Meeting(matter)',
                    'created_at',
                    'Had Vision',
                   'Had Strategy',
                   'Had Discovery',
                   'First Lead',
                    'ID',
                    'Date Created',
                    'Max Fee',
                    'Leadsource']

# Drop all other columns
FINAL_TABLE = merged_df_FINAL_copy[columns_to_keep]

# Save the modified dataframe to an Excel file
FINAL_TABLE.to_excel('FINAL_TABLE.xlsx', index=False)